In [1]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
gc = gspread.service_account()

In [2]:
scopes = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
          "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file('secrets/linkml-336707-1c85304108ea.json', scopes=scopes)
client = gspread.authorize(creds)


In [3]:
import time
#time.sleep(1)

In [4]:
names = ["sr", "lc", "tl", "wd", "ST", "rc", "mw", "Ray S", "ad", "wd", "lr", "dd", "pfey", \
         "SB", "AA", "cjm", "js", "kb", "leila", "VW", "dolson", "aleix@ebi.ac.uk", "tr", "esegerd3", "pascale"]

In [5]:
dfs = []
all_rows = []
for name in names:
    print(name)
    wks = gc.open(str(f"Definition Evaluation Template - {name}")).worksheet("Definitions (EDIT HERE)")
    rows = wks.get_all_records()
    rows = [row for row in rows if row["score"] or row["accuracy"]]
    propagated = 0
    for row in rows:
        row["evaluator"] = name
        for k in ["accuracy", "internal_consistency", "score", "confidence"]:
            v = row[k]
            if v == "?":
                v = ""
            if isinstance(v,float):
                v = round(v)
            if isinstance(v,str) and "-" in v:
                # some customized their form
                v = int(v.split("-")[0])
            if v and v not in list(range(1,6)):
                print(f"BAD VAL {v} in {row}")
                row[k] = None
        if not row["score"] and row["accuracy"]:
            propagated += 1
            row["score"] = row["accuracy"]
    all_rows.extend(rows)
    wks_df = pd.DataFrame(rows)
    print("rows", len(rows))
    print("propagated", propagated)
    wks_df["evaluator"] = name
    time.sleep(3)
    
    dfs.append(wks_df)
#eval_df = pd.concat(dfs)
eval_df = pd.DataFrame(all_rows)
eval_df.to_csv("definitions-sheets/combined.csv", index=False)
eval_df["evaluator"].unique()

sr
rows 700
propagated 2
lc
BAD VAL   in {'ontology': 'hp', 'label': 'Bent sperm neck', 'definition': 'A structural abnormality in which the sperm neck is bent or curved.', 'internal_id': 1515, 'accuracy': ' ', 'internal_consistency': 5, 'score': 5, 'confidence': 5, 'notes': '', 'is_a': 'AbnormalSpermNeckMorphology', 'relationships': '', 'evaluator': 'lc'}
rows 558
propagated 0
tl
rows 102
propagated 0
wd
rows 315
propagated 0
ST
rows 644
propagated 0
rc
rows 245
propagated 0
mw
rows 245
propagated 0
Ray S
rows 362
propagated 0
ad
rows 53
propagated 0
wd
rows 315
propagated 0
lr
rows 36
propagated 3
dd
rows 243
propagated 0
pfey
rows 27
propagated 0
SB
rows 75
propagated 0
AA
rows 190
propagated 0
cjm
rows 486
propagated 0
js
rows 288
propagated 0
kb
rows 245
propagated 0
leila
rows 164
propagated 0
VW
rows 215
propagated 215
dolson
rows 0
propagated 0
aleix@ebi.ac.uk
rows 435
propagated 0
tr
rows 133
propagated 0
esegerd3
rows 147
propagated 0
pascale
rows 123
propagated 0


array(['sr', 'lc', 'tl', 'wd', 'ST', 'rc', 'mw', 'Ray S', 'ad', 'lr',
       'dd', 'pfey', 'SB', 'AA', 'cjm', 'js', 'kb', 'leila', 'VW',
       'aleix@ebi.ac.uk', 'tr', 'esegerd3', 'pascale'], dtype=object)

In [6]:
eval_df.groupby(["ontology", "evaluator"]).size()

ontology  evaluator      
cl        ad                  53
          aleix@ebi.ac.uk    435
          cjm                173
          esegerd3           147
          lc                 145
          lr                  36
          pfey                27
          sr                 432
          tl                 102
          wd                  54
envo      js                 288
          leila              164
          wd                 576
foodon    dd                 243
          kb                 245
          mw                 245
          rc                 245
          tr                 133
go        Ray S              236
          ST                 230
          VW                 215
          pascale            123
hp        ST                 178
          lc                 177
mondo     ST                 236
          lc                 236
mp        AA                 190
          SB                  75
oba       Ray S              126
          cjm    

In [7]:
src_df = pd.read_csv("definitions-sheets/key.tsv", sep="\t")
rows = []
for _, row in src_df.iterrows():
    if row["is_original"] == True:
        row["method"] = "curator"
        row["model_name"] = "human"
    else:
        if row["model_name"] == "nous-hermes-13b" and row["method"] != "direct":
            # too few
            continue
        if row["method"] == "direct":
            row["method"] = "RAG"
        elif row["method"] == "background":
            row["method"] = "RAG+background"
        elif row["method"] == "github":
            row["method"] = "RAG+github"
        elif row["method"] == "devdocs":
            row["method"] = "RAG+docs"
    rows.append(row)
src_df = pd.DataFrame(rows)

df = pd.merge(eval_df, src_df, how='inner', on='definition')

In [8]:
src_df["is_original"].unique()

array([False,  True])

In [9]:
df.groupby(["method", "model_name"]).size()

method          model_name     
RAG             gpt-3.5-turbo      1051
                gpt-4              1069
                nous-hermes-13b    1025
RAG+background  gpt-3.5-turbo      1063
                gpt-4               961
RAG+docs        gpt-3.5-turbo       189
RAG+github      gpt-3.5-turbo       399
                gpt-4               397
curator         human               938
no_RAG          gpt-4               145
dtype: int64

In [10]:
for k in ["score_x", "accuracy_x", "internal_consistency_x"]:
    df[k] = pd.to_numeric(df[k], errors='coerce')
